In [1]:
import torch
import torchvision.models as models
import torchvision.transforms as transform
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, WeightedRandomSampler
import torch.optim as optim
import torch.nn as nn
import numpy as np

In [2]:
class CustomAugmentedDataset(ImageFolder):
    
    # root: folder dataset, transform_dict: nama class & augmentasi khusus
    def __init__(self, root, transform_dict, default_transform=None):
        super().__init__(root)
        self.transform_dict = transform_dict
        self.default_transform = default_transform
    
    # ambil gambar dari class, apply augmentasi, return gambar hasil augmentasi
    def __getitem__(self, index):
        path, target = self.samples[index]
        sample = self.loader(path)
    
        class_name = self.classes[target]
        transform = self.transform_dict.get(class_name, self.default_transform)

        if transform:
            sample = transform(sample)
        
        return sample, target

In [3]:
default_transform = transform.Compose([
    transform.Resize(256),
    transform.CenterCrop(224),
    transform.ToTensor(),
    transform.Normalize(mean=[0.485, 0.456, 0.406],
                 std=[0.229, 0.224, 0.225]),
])

strong_transform = transform.Compose([
    transform.Resize(256),
    transform.RandomResizedCrop(224),
    transform.RandomHorizontalFlip(),
    transform.RandomRotation(30),
    transform.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transform.ToTensor(),
    transform.Normalize(mean=[0.485, 0.456, 0.406],
                 std=[0.229, 0.224, 0.225]),
])

In [4]:
# deteksi augmentasi khusus buat red spot
small_classes = ['red-spot']
transform_dict = {cls: strong_transform for cls in small_classes}

# load dataset beserta transform-nya
train_dataset = CustomAugmentedDataset(root='../../dataset/Train/', transform_dict=transform_dict, default_transform=default_transform)
val_dataset = CustomAugmentedDataset(root='../../dataset/Valid/', transform_dict=transform_dict, default_transform=default_transform)

# class count
class_counts = np.bincount([label for _, label in train_dataset.samples])
class_weights = 1. / torch.tensor(class_counts, dtype=torch.float)

# sampler buat ngatasin class imbalance dengan cara ngasih probabilitas sampling lebih tinggi ke class yang punya data sedikit
sample_weights = [class_weights[label] for _, label in train_dataset.samples]
sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

# dataloader
train_dataloader = DataLoader(train_dataset, batch_size=8, sampler=sampler, num_workers=0, pin_memory=True)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=0)

print(f"Classes detected: {train_dataset.classes}")

print("Augmentation summary per class:")
for cls in train_dataset.classes:
    print(f"{cls.ljust(15)} → {'Strong' if cls in transform_dict else 'Default'}")

Classes detected: ['algal_spot', 'brown-blight', 'gray-blight', 'healthy', 'helopeltis', 'leaf-rust', 'red-rust', 'red-spider-infested', 'red-spot', 'white-spot']
Augmentation summary per class:
algal_spot      → Default
brown-blight    → Default
gray-blight     → Default
healthy         → Default
helopeltis      → Default
leaf-rust       → Default
red-rust        → Default
red-spider-infested → Default
red-spot        → Strong
white-spot      → Default


In [5]:
# import model
model = models.densenet169(pretrained=True) # GANTI SESUAI MODEL YANG MAU DI-TRAIN
model.eval()

c:\Users\Raiza Rahman\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
c:\Users\Raiza Rahman\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=DenseNet169_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet169_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [6]:
num_classes = len(train_dataset.classes)

# ganti layer terakhir biar disesuain sama jumlah class di dataset
model.classifier = nn.Linear(model.classifier.in_features, num_classes) # ADJUST SESUAI MODEL
print(model.classifier.in_features)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('is cuda available?', torch.cuda.is_available())
model = model.to(device)


1664
is cuda available? True


In [7]:
# loss function buat klasifikasi multi-class
criterion = nn.CrossEntropyLoss()

# optimizer
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.00001)

In [8]:
num_epochs = 30

print(f"Total batches: {len(train_dataloader)}")
best_val_loss = float('inf')

for epoch in range(num_epochs):
    print(f"Starting Epoch {epoch+1}/{num_epochs}")
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (images, labels) in enumerate(train_dataloader):
        if batch_idx % 10 == 0:
            print(f"Processing batch {batch_idx+1}/{len(train_dataloader)}")
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    avg_loss = running_loss / len(train_dataloader)
    train_accuracy = (correct / total) * 100
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}, Accuracy: {train_accuracy:.2f}%")

    # validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in val_dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)

    avg_val_loss = val_loss / len(val_dataloader)
    val_accuracy = (val_correct / val_total) * 100
    print(f"Validation Loss: {avg_val_loss:.4f}, Accuracy: {val_accuracy:.2f}%")

Total batches: 1319
Starting Epoch 1/30
Processing batch 1/1319
Processing batch 11/1319
Processing batch 21/1319
Processing batch 31/1319
Processing batch 41/1319
Processing batch 51/1319
Processing batch 61/1319
Processing batch 71/1319
Processing batch 81/1319
Processing batch 91/1319
Processing batch 101/1319
Processing batch 111/1319
Processing batch 121/1319
Processing batch 131/1319
Processing batch 141/1319
Processing batch 151/1319
Processing batch 161/1319
Processing batch 171/1319
Processing batch 181/1319
Processing batch 191/1319
Processing batch 201/1319
Processing batch 211/1319
Processing batch 221/1319
Processing batch 231/1319
Processing batch 241/1319
Processing batch 251/1319
Processing batch 261/1319
Processing batch 271/1319
Processing batch 281/1319
Processing batch 291/1319
Processing batch 301/1319
Processing batch 311/1319
Processing batch 321/1319
Processing batch 331/1319
Processing batch 341/1319
Processing batch 351/1319
Processing batch 361/1319
Processin

In [9]:
torch.save(model.state_dict(), "densenet169.pth")
print("Model saved successfully!")

Model saved successfully!
